In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

In [2]:
data = pd.read_csv('gold_mining_data.csv')
data

,dwarf_id,hired_date,reported_date,gold_mined,hammer_date
0,1,2014-01-01,2014-01-31,656,2014-01-01
1,1,2014-01-01,2014-02-28,565,2014-01-01
2,1,2014-01-01,2014-03-31,516,2014-01-01
3,1,2014-01-01,2014-04-30,568,2014-01-01
4,1,2014-01-01,2014-05-31,556,2014-01-01
...,...,...,...,...,...
13488,2673,2015-08-01,2015-08-31,695,2015-01-01
13489,2674,2015-08-01,2015-08-31,573,2015-01-01
13490,2675,2015-08-01,2015-08-31,617,2015-01-01
13491,2676,2015-08-01,2015-08-31,609,2015-01-01


Для удобства перекодируем классы молотов  
0 - 2014-01-01  
1 - 2015-01-01

In [12]:
from sklearn.preprocessing import LabelEncoder
model = LabelEncoder()
model.fit(data.hammer_date).classes_
hammers = model.transform(data.hammer_date)
data.hammer_date = hammers

Переводим даты начала работы и дату отчета в формат даты

In [3]:
data.hired_date = pd.to_datetime(data.hired_date)
data.reported_date = pd.to_datetime(data.reported_date)
data

,dwarf_id,hired_date,reported_date,gold_mined,hammer_date
0,1,2014-01-01,2014-01-31,656,2014-01-01
1,1,2014-01-01,2014-02-28,565,2014-01-01
2,1,2014-01-01,2014-03-31,516,2014-01-01
3,1,2014-01-01,2014-04-30,568,2014-01-01
4,1,2014-01-01,2014-05-31,556,2014-01-01
...,...,...,...,...,...
13488,2673,2015-08-01,2015-08-31,695,2015-01-01
13489,2674,2015-08-01,2015-08-31,573,2015-01-01
13490,2675,2015-08-01,2015-08-31,617,2015-01-01
13491,2676,2015-08-01,2015-08-31,609,2015-01-01


In [4]:
def time(row):
    row['production'] = (row.reported_date - row.hired_date)
    return row

Преобразуем дату из формата timedelta64 в "месяцы"

In [5]:
data = data.apply(time, axis=1)
data.production = data.production/np.timedelta64(1, 'M')
data

,dwarf_id,hired_date,reported_date,gold_mined,hammer_date,production
0,1,2014-01-01,2014-01-31,656,2014-01-01,0.985647
1,1,2014-01-01,2014-02-28,565,2014-01-01,1.905583
2,1,2014-01-01,2014-03-31,516,2014-01-01,2.924085
3,1,2014-01-01,2014-04-30,568,2014-01-01,3.909731
4,1,2014-01-01,2014-05-31,556,2014-01-01,4.928233
...,...,...,...,...,...,...
13488,2673,2015-08-01,2015-08-31,695,2015-01-01,0.985647
13489,2674,2015-08-01,2015-08-31,573,2015-01-01,0.985647
13490,2675,2015-08-01,2015-08-31,617,2015-01-01,0.985647
13491,2676,2015-08-01,2015-08-31,609,2015-01-01,0.985647


С помощью сводной таблицы группируем отдельно записи каждого гнома с учетом использования конкретного  
орудия труда, определяем среднюю производительность труда

In [47]:
pivot = pd.pivot_table(data, index=['dwarf_id', 'hammer_date'], values=['gold_mined', 'production'], aggfunc=sum)
pivot['prod_per_month'] = pivot.gold_mined / pivot.production
pivot = pivot.astype(int)
pivot

,,gold_mined,production,prod_per_month
dwarf_id,hammer_date,,,
1,0,10072,208,48
2,0,10065,208,48
3,0,9898,208,47
4,0,10050,208,48
5,0,10136,208,48
...,...,...,...,...
2673,1,695,0,705
2674,1,573,0,581
2675,1,617,0,625
